In [ ]:
import os
import json
import torch
import numpy as np
import cv2
import pandas as pd
from time import time
from ultralytics import YOLO
import supervision as sv

In [ ]:
bug_model = '/home/farzam/workspace/python/ml/Courses/TRACO/junkTest/runs/detect/train2/weights/best.pt'

In [1]:
class ObjectDetection:

    def __init__(self, capture_index):

        self.capture_index = capture_index

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        print("Using Device: ", self.device)

        self.model = self.load_model()

        self.CLASS_NAMES_DICT = self.model.model.names

        print(self.CLASS_NAMES_DICT)

        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=2, text_thickness=1, text_scale=1)

    def load_model(self):

        model = YOLO(bug_model)
        model.fuse()

        return model

    def predict(self, frame):

        results = self.model(frame)

        return results

    def plot_bboxes(self, results, frame):

        xyxys = []
        confidences = []
        class_ids = []

        # Extract detections for person class
        for result in results:
            boxes = result.boxes.cpu().numpy()
            if len(boxes) > 0:
                class_id = boxes.cls[0]
                conf = boxes.conf[0]
                xyxy = boxes.xyxy[0]

                if class_id == 0.0:
                    xyxys.append(result.boxes.xyxy.cpu().numpy())
                    confidences.append(result.boxes.conf.cpu().numpy())
                    class_ids.append(result.boxes.cls.cpu().numpy().astype(int))

        # Setup detections for visualization
        detections = sv.Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int),
        )

        # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
                       for _, _, confidence, class_id, tracker_id
                       in detections]

        # Annotate and display frame
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)

        return frame

    def __call__(self):

        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        while True:

            start_time = time()

            ret, frame = cap.read()
            assert ret

            results = self.predict(frame)
            frame = self.plot_bboxes(results, frame)

            end_time = time()
            fps = 1 / np.round(end_time - start_time, 2)

            cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

            cv2.imshow('YOLOv8 Detection', frame)

            if cv2.waitKey(5) & 0xFF == 27:
                break

        cap.release()
        cv2.destroyAllWindows()

In [2]:
detector = ObjectDetection(capture_index=0)

NameError: name 'torch' is not defined

In [ ]:
# detector()

In [3]:
if not os.path.exists('cropped_bugs'):
    os.mkdir('cropped_bugs')

data = pd.DataFrame(columns=['CroppedHexBugCoordinationX', 'CroppedHexBugCoordinationY',
                             'OriginalBoxCoordinationX1', 'OriginalBoxCoordinationX2',
                             'OriginalBoxCoordinationY1', 'OriginalBoxCoordinationY2', 'Path', 'ID'])

NameError: name 'os' is not defined

In [4]:
max_height = 300
max_width = 300

for directory_name in os.listdir('./samples'):
    directory_ID = directory_name[8:]
    for sample_file_name in os.listdir(f'./samples/{directory_name}'):
        frame_id = int(sample_file_name.split('.')[0][5:])
        print('------------------------------------------------------------------')
        print(f'Processing file {directory_name}/{sample_file_name}')

        sample_path = f'./samples/{directory_name}/{sample_file_name}'
        sample_img = cv2.imread(sample_path)

        prediction_results = detector.predict(sample_img)

        for result in prediction_results:
            xyxy = None
            crop_img = None

            boxes = result.boxes.cpu().numpy()
            if len(boxes) > 0:
                xyxy = list(map(int, boxes.xyxy[0]))

                crop_img = sample_img[xyxy[1]: xyxy[3], xyxy[0]: xyxy[2]]

                cropped_img_shape = crop_img.shape

                height_margin = max_height - cropped_img_shape[0]
                width_margin = max_width - cropped_img_shape[1]

                if xyxy[2] + width_margin > sample_img.shape[1]:
                    x1, x2 = xyxy[0] - width_margin, xyxy[2]
                else:
                    x1, x2 = xyxy[0], xyxy[2] + width_margin

                if xyxy[3] + height_margin > sample_img.shape[0]:
                    y1, y2 = xyxy[1] - height_margin, xyxy[3]
                else:
                    y1, y2 = xyxy[1], xyxy[3] + height_margin

                margined_crop_img = sample_img[y1: y2, x1: x2]

                if margined_crop_img is not None:
                    if not os.path.exists(f'./cropped_bugs/{directory_name}'):
                        os.mkdir(f'./cropped_bugs/{directory_name}')

                    cv2.imwrite(f'./cropped_bugs/{directory_name}/{sample_file_name}', margined_crop_img)

                    with open(f'../Annotations/training{directory_ID}.traco') as file:
                        sample_ann = json.load(file)

                    x, y = int(sample_ann['rois'][frame_id]['pos'][0]), int(sample_ann['rois'][frame_id]['pos'][1])

                    new_x, new_y = x - x1, y - y1

                    new_row = pd.DataFrame(
                        {'CroppedHexBugCoordinationX': new_x,
                         'CroppedHexBugCoordinationY': new_y,
                         'OriginalBoxCoordinationX1': x1,
                         'OriginalBoxCoordinationX2': x2,
                         'OriginalBoxCoordinationY1': y1,
                         'OriginalBoxCoordinationY2': y2,

                         'Path': f'cropped_bugs/{directory_name}/{sample_file_name}',
                         'ID': frame_id},
                        index=[0])

                    data = pd.concat([new_row, data.iloc[:]]).reset_index(drop=True)


NameError: name 'os' is not defined

In [ ]:
# print(data)
data.to_csv('data.csv')